   # Introduction

Twitter is one of the most popular social media platforms and a rich source of data for analyzing social trends and user behavior.

With the help of Neo4j, a powerful graph database, we can easily represent Twitter data as a graph and perform complex queries to gain insights into the data.

In this tutorial, we will go through the process of setting up Neo4j, importing Twitter data into Neo4j using Python, and performing some basic queries on the data.

Population of the Neo4j graph model with the tweets that have been collected in Python

### Install Required Packages

Before we dive into the code, let's talk a little bit about Neo4j. Neo4j is a graph database that is optimized for storing and querying graph data. It is particularly useful for scenarios where relationships between data points are just as important as the data points themselves. For example, if you were building a social network, the relationships between users (e.g., friends, followers, etc.) would be just as important as the users themselves.

To get started, we need to install a few packages that we will be using in our Python code. Specifically, we will be using the neo4j python library. These packages will allow us to interact with Neo4j and perform any queries we want. Furthermore we need an active (local) Neo4j Graph DB instance to be active.

Here's the code block to install these packages:

In [1]:
# !pip install neo4j

Here, we are installing the following package:

- neo4j: This is another Python package that provides a driver for interacting with Neo4j databases.

Now that we have installed the required packages, we can move on to the next step.

### Import libraries

Now we need to import the necessary packages for our Python script to interact with Neo4j. Specifically, we're importing:

- json: a package for working with JSON data
- numpy: a package for scientific computing with Python
- pandas: a package for data manipulation and analysis
- neo4j: a package for connecting to a Neo4j database

We'll use these packages to load data from a JSON file, create nodes and relationships in Neo4j, and perform queries on the database.

In [2]:
import json

import numpy as np
import pandas as pd

from neo4j import GraphDatabase

### Open JSON file with python

Let's open the JSON file, WDM1.json, using the json package and reads its contents into the data variable. The encoding parameter is set to 'utf-8' to ensure that the data is properly decoded from the file. We can assume that the JSON file contains data in a specific format that we will use to create nodes and relationships in Neo4j.

In [3]:
# tweets_json = 'C:\\Users\\Greg\\Documents\\vakali\\WDM1.json'
tweets_json = 'C:\\Users\\costa\\Documents\\MSc assignments\\Web Mining\\1st Lab Project - Neo4j\\wdm1_clean.json'

with open(tweets_json, encoding='utf-8') as data_file:
    data = json.load(data_file)

### Clean Up of the data

The below code is deleting the context_annotations key from the tweets in the data list.

In [5]:
for tweet in data:
    try:
        del tweet['includes']['tweets'][0]['context_annotations']
    except:
        continue
        # No context annotations in tweet 0
        
    try:
        del tweet['includes']['tweets'][1]['context_annotations']
    except:
        continue
        # No context annotations in tweet 1    

The below code is creating a dictionary for each of the nodes and relationships that we will be using in our graph.

Specifically it creates five Python dictionaries (`users_dict`, `tweets_dict`, `url_dict`, `hashtag_dict`, and `user_tweet_relationship`) that will be used to store information about Twitter users, their tweets, and relationships between them.

The `users_dict` dictionary contains information about Twitter users, including their user ID, username, and number of followers. The `tweets_dict` dictionary contains information about tweets, including the tweet ID, text, impression count, date, and number of replies. The `url_dict` dictionary contains URLs that have been used in tweets, while the `hashtag_dict` dictionary contains hashtags that have been used in tweets.

Finally, the `user_tweet_relationship` dictionary contains information about relationships between Twitter users and their tweets, while the `user_mentioned_relationship`, `user_used_hashtag`, `user_used_url`, `tweet_has_url`, and `tweet_has_hashtag` dictionaries contain information about other types of relationships between users, tweets, and various entities like hashtags and URLs.

These dictionaries will be useful for storing information about the data extracted from the JSON file, and they will be used later to create nodes and relationships in the Neo4j database.

In [6]:
users_dict = {'user_id': [],
             'username': [],
             'followers_count': []}

tweets_dict = {'tweet_id': [],
              'text': [],
              'impression_count': [],
              'tweet_date': [],
              'reply_count': []}

url_dict = {'url': []}

hashtag_dict = {'hashtag': []}

user_tweet_relationship = {'user_id': [],
                            'tweet_id': [],
                          'type': [],
                          'date': []}

user_mentioned_relationship = {'user_id': [],
                               'mentioned_user_id': [],
                               'in': []}

user_used_hashtag = {'user_id': [],
                    'hashtag': [],
                    'in': []}

user_used_url = {'user_id': [],
                'url': [],
                'in': []}

tweet_has_url = {'tweet_id': [],
                'url': []}

tweet_has_hashtag = {'tweet_id': [],
                    'hashtag': []}

Updating the dictionaries with the data from the tweets.

This is a Python function called `update_dictionary` which takes two arguments, `dictionary_to_be_updated` and `data`. The purpose of this function is to update a specific dictionary based on the given data.

There are 10 different dictionaries that can be updated using this function, each one corresponding to a different type of data. Depending on which dictionary is being updated, the function will append the data to the appropriate keys in the dictionary.

For example, if `dictionary_to_be_updated` is `'users_dict'`, the function will append the `user_id`, `username`, and `followers_count` values from the data argument to their respective keys in the `users_dict` dictionary.

Overall, this function is a useful tool for organizing the data from the provided JSON file into separate dictionaries, each corresponding to a different type of information. This will make it easier to create nodes and relationships in the Neo4j database later on.

In [7]:
def update_dictionary(dictionary_to_be_updated, data):
    if dictionary_to_be_updated == 'users_dict':
        users_dict['user_id'].append(data[0])
        users_dict['username'].append(data[1])
        users_dict['followers_count'].append(data[2])
    
    elif dictionary_to_be_updated == 'tweets_dict':
        tweets_dict['tweet_id'].append(data[0])
        tweets_dict['text'].append(data[1])
        tweets_dict['impression_count'].append(data[2])
        tweets_dict['tweet_date'].append(data[3])
        tweets_dict['reply_count'].append(data[4])
    
    elif dictionary_to_be_updated == 'url_dict':
        url_dict['url'].append(data[0])
    
    elif dictionary_to_be_updated == 'hashtag_dict':
        hashtag_dict['hashtag'].append(data[0])
    
    elif dictionary_to_be_updated == 'user_tweet_relationship':
        user_tweet_relationship['user_id'].append(data[0])
        user_tweet_relationship['tweet_id'].append(data[1])
        user_tweet_relationship['type'].append(data[2])
        user_tweet_relationship['date'].append(data[3])
        
    elif dictionary_to_be_updated == 'user_mentioned_relationship':
        user_mentioned_relationship['user_id'].append(data[0])
        user_mentioned_relationship['mentioned_user_id'].append(data[1])
        user_mentioned_relationship['in'].append(data[2])
    
    elif dictionary_to_be_updated == 'user_used_hashtag':
        user_used_hashtag['user_id'].append(data[0])
        user_used_hashtag['hashtag'].append(data[1])
        user_used_hashtag['in'].append(data[2])
    
    elif dictionary_to_be_updated == 'user_used_url':
        user_used_url['user_id'].append(data[0])
        user_used_url['url'].append(data[1])
        user_used_url['in'].append(data[2])
        
    elif dictionary_to_be_updated == 'tweet_has_url':
        tweet_has_url['tweet_id'].append(data[0])
        tweet_has_url['url'].append(data[1])
        
    elif dictionary_to_be_updated == 'tweet_has_hashtag':
        tweet_has_hashtag['tweet_id'].append(data[0])
        tweet_has_hashtag['hashtag'].append(data[1])

Now we need to parse through a dataset of Twitter data and extract various information about the users, tweets and relationships between them.

The below code first initializes some dictionaries that will be used to store information about the users, tweets, and relationships between them. Then, it iterates through each tweet in the dataset and extracts information about the user who posted the tweet and the tweet itself. It also checks to see if the tweet is an original tweet or a retweet by looking at the 'referenced_tweets' section. If it is a retweet, it extracts information about the referenced tweet as well.

Next, the script checks if the tweet has any mentions, urls, or hashtags in its 'entities' section. If it does, it extracts information about them and updates the appropriate dictionaries.

Finally, if the tweet has a referenced user and tweet (i.e., it is a retweet), the script extracts information about the referenced user and tweet, updates the appropriate dictionaries, and adds a relationship between the user who posted the original tweet and the referenced user.

In [8]:
for tweet in data:
    
    # User 0 information - Tweet author info (tweet/reply/quote/retweet)
    user_id_0 = tweet['includes']['users'][0]['id']
    username_0 = tweet['includes']['users'][0]['username']
    user_followers_count_0 = tweet['includes']['users'][0]['public_metrics']['followers_count']

    update_dictionary('users_dict', [user_id_0, username_0, user_followers_count_0])
    
    # Tweet 0 information - Info on the tweet that is referenced in User0.
    tweet_id_0 = tweet['includes']['tweets'][0]['id']
    tweet_text_0 = tweet['includes']['tweets'][0]['text']
    tweet_impression_count_0 = tweet['includes']['tweets'][0]['public_metrics']['impression_count']
    tweet_date_0 = tweet['includes']['tweets'][0]['created_at']
    tweet_replies_0 = tweet['includes']['tweets'][0]['public_metrics']['reply_count']
    update_dictionary('tweets_dict', [tweet_id_0, tweet_text_0, tweet_impression_count_0, tweet_date_0, tweet_replies_0])
        
    # if there is a reference, obtain its type (reply/quote/retweet), otherwise it is an original tweet
    if ('referenced_tweets' in tweet['includes']['tweets'][0]):
        for reference in tweet['includes']['tweets'][0]['referenced_tweets']:
            tweet_0_type = reference['type']
            update_dictionary('user_tweet_relationship', [user_id_0, tweet_id_0, tweet_0_type, tweet_date_0])
    else:
        tweet_0_type = 'tweeted'
        update_dictionary('user_tweet_relationship', [user_id_0, tweet_id_0, tweet_0_type, tweet_date_0])

    # Not all tweets have the 'entities' section (info on mentions, urls hashtags, if any)
    if ('entities' in tweet['includes']['tweets'][0]):
        
        if ('mentions' in tweet['includes']['tweets'][0]['entities']):
            for mention in tweet['includes']['tweets'][0]['entities']['mentions']:
                update_dictionary('user_mentioned_relationship', [user_id_0, mention['id'], tweet_id_0])
                
        # Not all entities contain the 'urls' section
        if ('urls' in tweet['includes']['tweets'][0]['entities']):
            for url in tweet['includes']['tweets'][0]['entities']['urls']:
                update_dictionary('url_dict', [url['expanded_url']])
                update_dictionary('tweet_has_url', [tweet_id_0, url['expanded_url']])
                update_dictionary('user_used_url', [user_id_0, url['expanded_url'], tweet_id_0])
        
        # Not all entities contain the 'hashtags' section
        if ('hashtags' in tweet['includes']['tweets'][0]['entities']):
            for hashtag in tweet['includes']['tweets'][0]['entities']['hashtags']:
                update_dictionary('hashtag_dict', [hashtag['tag']])
                update_dictionary('tweet_has_hashtag', [tweet_id_0, hashtag['tag']])
                update_dictionary('user_used_hashtag', [user_id_0, hashtag['tag'], tweet_id_0])
        
    # If there is a referenced object (i.e. user[1] and tweet[1]) extract all info as before
    if len(tweet['includes']['users']) > 1 and len(tweet['includes']['tweets']) > 1:

        # Extract User 1 node information
        user_id_1 = tweet['includes']['users'][1]['id']
        username_1 = tweet['includes']['users'][1]['username']
        user_followers_count_1 = tweet['includes']['users'][1]['public_metrics']['followers_count']
        
        update_dictionary('users_dict', [user_id_1, username_1, user_followers_count_1])
        
        # Extract User 1 tweet information
        tweet_id_1 = tweet['includes']['tweets'][1]['id']
        tweet_text_1 = tweet['includes']['tweets'][1]['text']
        tweet_impression_count_1 = tweet['includes']['tweets'][1]['public_metrics']['impression_count']
        tweet_date_1 = tweet['includes']['tweets'][1]['created_at']
        tweet_replies_1 = tweet['includes']['tweets'][1]['public_metrics']['reply_count']
     
        update_dictionary('tweets_dict', [tweet_id_1, tweet_text_1, tweet_impression_count_1, tweet_date_1, tweet_replies_1])       
        
        update_dictionary('user_mentioned_relationship', [user_id_0, user_id_1, tweet_id_0])
        
        # if there is a reference, obtain its type. otherwise it is an original tweet
        if ('referenced_tweets' in tweet['includes']['tweets'][1]):
            for reference in tweet['includes']['tweets'][1]['referenced_tweets']:
                tweet_1_type = reference['type']
                update_dictionary('user_tweet_relationship', [user_id_1, tweet_id_1, tweet_1_type, tweet_date_1])
        else:
            tweet_1_type = 'tweeted'
            update_dictionary('user_tweet_relationship', [user_id_1, tweet_id_1, tweet_1_type, tweet_date_1])        
    
        # Not all tweets have the 'entities' section
        if ('entities' in tweet['includes']['tweets'][1]):
            
            if ('mentions' in tweet['includes']['tweets'][1]['entities']):
                for mention in tweet['includes']['tweets'][1]['entities']['mentions']:
                    update_dictionary('user_mentioned_relationship', [user_id_1, mention['id'], tweet_id_1])
            
            # Not all entities contain the 'urls' section
            if ('urls' in tweet['includes']['tweets'][1]['entities']):
                for url in tweet['includes']['tweets'][1]['entities']['urls']:
                    update_dictionary('url_dict', [url['expanded_url']])
                    update_dictionary('tweet_has_url', [tweet_id_1, url['expanded_url']])
                    update_dictionary('user_used_url', [user_id_1, url['expanded_url'], tweet_id_1])
        
            # Not all entities contain the 'hashtags' section
            if ('hashtags' in tweet['includes']['tweets'][1]['entities']):
                for hashtag in tweet['includes']['tweets'][1]['entities']['hashtags']:
                    update_dictionary('hashtag_dict', [hashtag['tag']])
                    update_dictionary('tweet_has_hashtag', [tweet_id_1, hashtag['tag']])
                    update_dictionary('user_used_hashtag', [user_id_1, hashtag['tag'], tweet_id_1])

### User node

In [9]:
users_df = pd.DataFrame(users_dict)

# Drop any potential duplicate values 
users_df = users_df.drop_duplicates(subset=['user_id'], keep='first').reset_index(drop=True)
users_df.head()

,user_id,username,followers_count
0,1336660338289340416,Traceyb38701240,2880
1,3396828819,1OFFGINGER,5568
2,16060248,baski_LA,31165
3,1403861754808049666,Khulood_Almani,55457
4,2385598159,Kimnology,940


### Tweet Node

In [10]:
tweets_df = pd.DataFrame(tweets_dict)

# Drop any potential duplicate values
tweets_df = tweets_df.drop_duplicates(subset=['tweet_id'], keep='first').reset_index(drop=True)
tweets_df.head()

,tweet_id,text,impression_count,tweet_date,reply_count
0,1609340258868686850,RT @1OFFGINGER: Epic Christmas surprise just a...,0,2023-01-01T00:06:32.000Z,0
1,1605213170229923841,Epic Christmas surprise just arrived🤩 💃courtes...,2171,2022-12-20T14:46:57.000Z,2
2,1609339028989673476,RT @Khulood_Almani: ⚡️Internet of Everything (...,0,2023-01-01T00:01:39.000Z,0
3,1609138211493711878,⚡️Internet of Everything (#IoE) in Context\n\n...,16356,2022-12-31T10:43:40.000Z,17
4,1609340607385714690,RT @femtechconf: Excited to welcome you to #Fe...,0,2023-01-01T00:07:55.000Z,0


### URL node

In [11]:
url_df = pd.DataFrame(url_dict)

# Drop any potential duplicate values
url_df = url_df.drop_duplicates(keep='first').reset_index(drop=True)
url_df.head()

,url
0,https://twitter.com/1OFFGINGER/status/16052131...
1,https://twitter.com/WeAreTechWomen/status/1605...
2,https://twitter.com/Khulood_Almani/status/1609...
3,https://femtechconf.com
4,https://twitter.com/darrenplayfordn/status/160...


### Hashtag node

In [12]:
hashtags_df = pd.DataFrame(hashtag_dict)

# Drop any potential duplicate values
hashtags_df = hashtags_df.drop_duplicates(keep='first').reset_index(drop=True)
hashtags_df.head()

,hashtag
0,TechMas
1,12DaysOfTechMas
2,womenintech
3,tuesdayvibe
4,IoE


### User tweeted relationship

In [13]:
user_tweet = pd.DataFrame(user_tweet_relationship)

# Drop any potential duplicate values
user_tweet = user_tweet.drop_duplicates(keep='first').reset_index(drop=True)
user_tweet.head()

,user_id,tweet_id,type,date
0,1336660338289340416,1609340258868686850,retweeted,2023-01-01T00:06:32.000Z
1,3396828819,1605213170229923841,quoted,2022-12-20T14:46:57.000Z
2,16060248,1609339028989673476,retweeted,2023-01-01T00:01:39.000Z
3,1403861754808049666,1609138211493711878,tweeted,2022-12-31T10:43:40.000Z
4,2385598159,1609340607385714690,retweeted,2023-01-01T00:07:55.000Z


### User used hashtag relationship

In [14]:
user_used_hashtag_df = pd.DataFrame(user_used_hashtag)

# Drop any potential duplicate values
user_used_hashtag_df = user_used_hashtag_df.drop_duplicates(keep='first').reset_index(drop=True)
user_used_hashtag_df.head()

,user_id,hashtag,in
0,1336660338289340416,TechMas,1609340258868686850
1,1336660338289340416,12DaysOfTechMas,1609340258868686850
2,1336660338289340416,womenintech,1609340258868686850
3,1336660338289340416,tuesdayvibe,1609340258868686850
4,3396828819,TechMas,1605213170229923841


### Tweet has hashtag relationship

In [15]:
tweet_has_hashtag_df = pd.DataFrame(tweet_has_hashtag)

# Drop any potential duplicate values
tweet_has_hashtag_df = tweet_has_hashtag_df.drop_duplicates(keep='first').reset_index(drop=True)
tweet_has_hashtag_df.head()

,tweet_id,hashtag
0,1609340258868686850,TechMas
1,1609340258868686850,12DaysOfTechMas
2,1609340258868686850,womenintech
3,1609340258868686850,tuesdayvibe
4,1605213170229923841,TechMas


### User mentioned relationship

Simple mention, reply, retweet or quote

In [16]:
user_mentioned_df = pd.DataFrame(user_mentioned_relationship)

# Drop any potential duplicate values
user_mentioned_df = user_mentioned_df.drop_duplicates(keep='first').reset_index(drop=True)
user_mentioned_df

,user_id,mentioned_user_id,in
0,1336660338289340416,3396828819,1609340258868686850
1,1336660338289340416,3512757083,1609340258868686850
2,3396828819,3512757083,1605213170229923841
3,16060248,1403861754808049666,1609339028989673476
4,16060248,87151593,1609339028989673476
...,...,...,...
55409,1396652556735111175,3939574932,1614773926428549122
55410,893898343180533760,1463156480434081792,1614773968254423040
55411,1353637028785770497,1476513861356294144,1614773223094194183
55412,1353637028785770497,4876039539,1614773223094194183


### User used url relationship

In [17]:
user_used_url_df = pd.DataFrame(user_used_url)

# Drop any potential duplicate values
user_used_url_df = user_used_url_df.drop_duplicates(keep='first').reset_index(drop=True)
user_used_url_df.head()

,user_id,url,in
0,3396828819,https://twitter.com/1OFFGINGER/status/16052131...,1605213170229923841
1,3396828819,https://twitter.com/WeAreTechWomen/status/1605...,1605213170229923841
2,1403861754808049666,https://twitter.com/Khulood_Almani/status/1609...,1609138211493711878
3,1217483034888458240,https://femtechconf.com,1583015496600928259
4,1609335341718335488,https://twitter.com/darrenplayfordn/status/160...,1609340979076829185


### Tweet has url relationship

In [18]:
tweet_has_url_df = pd.DataFrame(tweet_has_url)

# Drop any potential duplicate values
tweet_has_url_df = tweet_has_url_df.drop_duplicates(keep='first').reset_index(drop=True)
tweet_has_url_df.head()

,tweet_id,url
0,1605213170229923841,https://twitter.com/1OFFGINGER/status/16052131...
1,1605213170229923841,https://twitter.com/WeAreTechWomen/status/1605...
2,1609138211493711878,https://twitter.com/Khulood_Almani/status/1609...
3,1583015496600928259,https://femtechconf.com
4,1609340979076829185,https://twitter.com/darrenplayfordn/status/160...


### Neo4j

#### Establish connection

The next step is to establish connection and create a session in python side with the neo4j database. To do so we used the following commands:

In [19]:
db_connection = GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "test123456"))
session = db_connection.session()

#### Clear Neo4j

Deleting all nodes and relationships in the database.

To be able to execute the target queries, first we need to initialize the database with the required nodes and relationships. First, we execute the following two commands in order to clean up the database from any previous data:

In [20]:
# session.run('match (a) -[r] -> () delete a, r')
# session.run('MATCH (n) DETACH DELETE n')

#### Create Nodes

Given the job which has taken place as described in the previous section, we proceed with the generation of the nodes and the relationships.

The below code is creating a list of statements that will be used to create nodes in the graph database.

In [21]:
# User nodes
for index, row in users_df.iterrows():
    create_user_statement = 'CREATE (u:User {{userid: \'{}\', username: \'{}\', followers_count: \'{}\'}})' \
        .format(row['user_id'], 
                row['username'], 
                row['followers_count'])
    session.run(create_user_statement)

# Hashtag nodes
for index, row in hashtags_df.iterrows():
    create_hashtag_statement = 'CREATE (h:Hashtag {{hashtag: \'{}\'}})'.format(row['hashtag'])
    session.run(create_hashtag_statement)

# URL nodes
for index, row in url_df.iterrows():
    create_url_statement = 'CREATE (u:URL {{url: \'{}\'}})'.format(row['url'])
    session.run(create_url_statement)

# Tweet nodes
for index, row in tweets_df.iterrows():
    query = "CREATE (t:Tweet {tweet_id: $id, tweet_text: $text, impression_count: $impressions, created_at: $created_at, reply_count: $reply_count})"
    parameters = {'id': row['tweet_id'], 
                  'text': row['text'], 
                  'impressions': row['impression_count'], 
                  'created_at': row['tweet_date'], 
                  'reply_count': row['reply_count']}
    session.run(query, parameters)

Through [neo4j browser](http://localhost:7474/browser/) it is easy to check if the nodes have been created as expected.

#### Create relationships

The next step in to create the relationships. 

In [22]:
# Tweet_has_url relationship
for index, row in tweet_has_url_df.iterrows():
    create_tweet_has_url_rel = \
    """MATCH (t:Tweet), (u:URL) WHERE t.tweet_id = \'{}\' AND u.url = \'{}\' CREATE (t)-[tu:HAS_URL]->(u) RETURN type(tu)""" \
    .format(row['tweet_id'], row['url'])
    session.run(create_tweet_has_url_rel)

# Tweet_has_hashtag relationship
for index, row in tweet_has_hashtag_df.iterrows():
    create_tweet_has_hashtag_rel = \
    """MATCH (t:Tweet), (h:Hashtag) WHERE t.tweet_id = \'{}\' AND h.hashtag = \'{}\' CREATE (t)-[th:HAS_HASHTAG]->(h) RETURN type(th)""" \
    .format(row['tweet_id'], row['hashtag'])
    session.run(create_tweet_has_hashtag_rel)

# User_used_url relationship
for index, row in user_used_url_df.iterrows():
    create_user_used_url_rel = \
    """MATCH (user:User), (u:URL) WHERE user.userid = \'{}\' AND u.url = \'{}\' CREATE (user)-[uu:USED_URL]->(u) RETURN type(uu)""" \
    .format(row['user_id'], row['url'])
    session.run(create_user_used_url_rel)

# user RETWEETED, REPLIED, QUOTED, TWEETED_ORIGINAL relationships
for index, row in user_tweet.iterrows():
    if row['type'] == 'retweeted':
        create_user_retweeted_rel = """MATCH (u:User), (t:Tweet) WHERE u.userid = \'{}\' AND t.tweet_id = \'{}\' CREATE (u)-[rt:RETWEETED]->(t) RETURN type(rt)""" \
            .format(row['user_id'], row['tweet_id'])
        session.run(create_user_retweeted_rel)
    elif row['type'] == 'replied_to':
        create_user_replied_rel = """MATCH (u:User), (t:Tweet) WHERE u.userid = \'{}\' AND t.tweet_id = \'{}\' CREATE (u)-[rp:REPLIED]->(t) RETURN type(rp)""" \
            .format(row['user_id'], row['tweet_id'])
        session.run(create_user_replied_rel)
    elif row['type'] == 'quoted':
        create_user_quoted_rel = """MATCH (u:User), (t:Tweet) WHERE u.userid = \'{}\' AND t.tweet_id = \'{}\' CREATE (u)-[qt:QUOTED]->(t) RETURN type(qt)""" \
            .format(row['user_id'], row['tweet_id'])
        session.run(create_user_quoted_rel)
    elif row['type'] == 'tweeted':
        create_user_tweeted_original_rel = """MATCH (u:User), (t:Tweet) WHERE u.userid = \'{}\' AND t.tweet_id = \'{}\' CREATE (u)-[to:TWEETED_ORIGINAL]->(t) RETURN type(to)""" \
            .format(row['user_id'], row['tweet_id'])
        session.run(create_user_tweeted_original_rel)

# user MENTIONED User relationship
for index, row in user_mentioned_df.iterrows():
    create_user_mentioned_rel = """MATCH (u0:User), (u1:User) WHERE u0.userid = \'{}\' AND u1.userid = \'{}\' CREATE (u0)-[mnt:MENTIONED]->(u1) RETURN type(mnt)""" \
            .format(row['user_id'], row['mentioned_user_id'])
    session.run(create_user_mentioned_rel)

# user USED_URL Url relationship
for index, row in user_used_hashtag_df.iterrows():
    create_user_used_hashtag_rel = \
    """MATCH (u:User), (h:Hashtag) WHERE u.userid = '{}' AND h.hashtag = '{}' CREATE (u)-[uh:USED_HASHTAG]->(h) RETURN type(uh)""" \
    .format(row['user_id'], row['hashtag'])
    session.run(create_user_used_hashtag_rel)


Just like the nodes, it is easy to check that relationships created as expected through [neo4j browser](http://localhost:7474/browser/).

### Queries

* 1) Get the total number of tweets.

* 3) Get the total number of hastags (case insensitive).

* 5) Get the 20 most popular URLs in descending order.

* 7) Get the followers count of each user.

* 9) Get the number of tweets & retweets per hour.

* 11) Get the user with the most replies.

* 13) Get the top-20 hashtags that co-occur with the hashtag that has been used the most.

* 15) Get the most "important" user in the dataset (user Graph algorithms: Pagerangk, Betweenness centrality, etc). You will apply these algorithms in the mention netwwork (which includes retweets).

* 17) For the 5th most important user, get the list of hashtags and URLs that have been posted (if no hashtags or URLs - check another user, e.g. 6th, 7th, etc...).

* 19) Get the user communities that have been created based on the users' interactions and visualize them (Louvain algorithm). (G P)

* Extra query 1: Get the top-10 users who posted the most tweets during the year 2022. In order to count a tweet, it should contain at least one of the top-5 mostly used hashtags of all time.

* Extra query 2: Get the users which have minimum average tweets per day bigger than the average number of tweets of a user per day. Only users with more that 2 tweets per day will be used in order to calculate the average number of tweets per day.

#### Query 1

##### Get the total number of tweets

Let's start with a warm up query to checkout the basic syntax. A tweet can be original, reply, retweet or quote. In the following query counts the actuals tweets which exists in the database:

In [23]:
query = 'MATCH (:User)-[:TWEETED_ORIGINAL]->(t:Tweet) RETURN count(t) as TotalTweets'
output = session.run(query)
print('Number of tweets: ', output.value())

Number of tweets:  [10043]


#### Query 3

##### Get the total number of hashtags (case insensitive)

In this query, the solutions required the usage of some functions. In order to achive the counting of unique hashtags the usage of count function and DISTINCT keyword is a must. Also, as the target result should be case insensitive, another function named "toLower" needs to be use to turn every hashtag to lower case. So, to sum up, the query which achieve the solutions can be shown below:

In [55]:
query = 'MATCH (h:Hashtag) \
         RETURN count(DISTINCT toLower(h.hashtag))'
output = session.run(query)
print('Number of hashtags (case insensitive): ', output.value())

Number of hashtags (case insensitive):  [12613]


#### Query 5

##### Get the 20 most popular URLs in descending order

Until now, the executed queries returned a single value. This is not the case with the query we have to answer now. Therefore, we need to figure out how to return mulitple values, or even a table as a result of our query.

Turns out the answer is quite simple: instead of using a single variable in the return clause, we can just add more variables. But in order to print them correctly, a change should also take place on the python side. The usage of "session.value()" command won't return the expected result as it will ignore the second variable. For this reason, the command "session.values()" should be used along with a for loop in order to have a neatly printed result. 

Also, to make the results more meaningful, we decided to use the "Order By" clause with "DESC" flag. This clause will order the results based on how many followers each user have in descending order. One detail here, is that the "toInteger" function should be used as "u.followers_count" is a string and the ordering is using the alphabetic order. So the final query can be found below:

In [25]:
query = 'MATCH (t:Tweet)-[:HAS_URL]->(u:URL) RETURN u.url, count(t) Order by count(t) desc limit 20'
output = session.run(query)
for record in output.values():
    print("URL '{}' has been used in {} tweets".format(record[0], record[1]))

URL 'https://fineartamerica.com' has been used in 119 tweets
URL 'https://twitter.com/conscientious1o/status/1612372707936718849/photo/1' has been used in 111 tweets
URL 'https://bit.ly/3WVRK58' has been used in 82 tweets
URL 'https://twitter.com/UNESCO/status/1611136758325846017/photo/1' has been used in 79 tweets
URL 'https://amzn.to/3jGb346' has been used in 62 tweets
URL 'https://womensframe.com/category/health-fitness/pregnancy/' has been used in 56 tweets
URL 'https://youtu.be/g5n1-hD-x5g' has been used in 56 tweets
URL 'https://amzn.to/3Q6es8q' has been used in 54 tweets
URL 'https://doi.org/10.1017/qpb.2021.18' has been used in 54 tweets
URL 'https://twitter.com/iamojon_/status/1611795133153804288/photo/1' has been used in 54 tweets
URL 'https://doi.org/10.1515/jib-2019-0005' has been used in 51 tweets
URL 'https://amzn.to/3Z5iMbZ' has been used in 50 tweets
URL 'http://bit.ly/GreenITNvidia' has been used in 49 tweets
URL 'https://twitter.com/Equal_Fights/status/161379150178756

#### Query 7

##### Get the followers count of each user

The query starts with the `MATCH` clause which specifies that we want to match all nodes labeled `User`. Here, `u` is an alias for the `User` node.

The `RETURN` clause specifies the properties of the `User` node that we want to return as output. Here, we want to return the `username`, `userid`, and `followers_count` properties of each `User` node.

The `as` keyword is used to provide aliases to the properties, which makes it easier to read and understand the output.

The result of the query is a table that contains the username, user ID, and followers count for each User node in the graph database.

The `output` variable is assigned the result of the query after it is executed using the `session.run()` method. The result is then converted into a Pandas DataFrame using the `pd.DataFrame()` method and assigned to the `followers_count_df` variable.

Overall, this query is useful for extracting basic information about users in a graph database and can be a starting point for more complex analysis and visualization tasks.

In [26]:
query = 'MATCH (u:User) RETURN toInteger(u.followers_count) as fc, u.username Order By fc desc'
output = session.run(query)
for val in output.values():
    print('User {} has {} followers'.format(val[1], val[0]))

User elonmusk has 128284304 followers
User narendramodi has 86388755 followers
User Twitter has 65457040 followers
User MileyCyrus has 46930973 followers
User deepikapadukone has 27242900 followers
User Reuters has 25737040 followers
User Forbes has 18710206 followers
User timesofindia has 14678419 followers
User UberFacts has 13620478 followers
User thekiranbedi has 12185146 followers
User TEDTalks has 11359357 followers
User sardesairajdeep has 8960481 followers
User Windows has 7794614 followers
User IndiaToday has 6246901 followers
User TheRealPCB has 5818033 followers
User Javedakhtarjadu has 4786676 followers
User wef has 4338609 followers
User FLOTUS has 4182244 followers
User amnestyusa has 4173233 followers
User UNHumanRights has 4022080 followers
User WorldBank has 3787717 followers
User Jemima_Khan has 3749271 followers
User inquirerdotnet has 3717912 followers
User UNESCO has 3675313 followers
User Naveen_Odisha has 3529236 followers
User ColorsTV has 3513392 followers
User

#### Query 9

##### Get the number of tweets & retweets per hour

This Neo4j query is composed of several parts that are combined using the `WITH` keyword. The query is used to retrieve data on the number of original tweets and retweets made by users during different hours of the day.

The first part of the query uses the `OPTIONAL MATCH` keyword to match users who have tweeted an original tweet with the `TWEETED_ORIGINAL` relationship, and then calculates the hour of the day using the datetime() function on the tweet's creation date. It then returns the hour, the number of tweets made during that hour, and 0 for the number of retweets. The results are then grouped by the hour and ordered in ascending order.

The second part of the query is similar to the first, but matches users who have retweeted a tweet with the `RETWEETED` relationship instead. It also calculates the hour of the day and returns the hour, 0 for the number of tweets, and the number of retweets made during that hour. The results are also grouped by the hour and ordered in ascending order.

The two parts of the query are combined using the `UNION ALL` keyword to create a single table of results that includes both the original tweets and the retweets. The results are then grouped again by the hour, and the number of tweets and retweets are summed up for each hour.

Finally, the last part of the query selects the hour, the number of tweets, and the number of retweets for each hour, and orders the results in ascending order by hour.

Overall, this query is useful for analyzing Twitter data and understanding how users interact with content on the platform during different hours of the day. The results can be used to identify patterns and trends in user behavior, which can be used to inform social media marketing strategies and content creation efforts.

In [54]:
query = """
    CALL {
    OPTIONAL MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet)
    WITH datetime(t.created_at).hour as hour, count(*) as tweets, 0 as retweets
    RETURN hour, sum(tweets) as tweets, sum(retweets) as retweets
    ORDER BY hour ASC

    UNION ALL

    OPTIONAL MATCH (u:User)-[:RETWEETED]->(t:Tweet)
    WITH datetime(t.created_at).hour as hour, 0 as tweets, count(*) as retweets
    RETURN hour, sum(tweets) as tweets, sum(retweets) as retweets
    ORDER BY hour ASC
    }
    WITH hour, sum(tweets) as tweets, sum(retweets) as retweets
    RETURN hour, tweets, retweets
    ORDER BY hour ASC
"""

# Hours in UTC time zone
output = session.run(query)
pd.DataFrame(output.data())

,hour,tweets,retweets
0,0,282,743
1,1,237,673
2,2,225,588
3,3,305,732
4,4,281,708
5,5,341,729
6,6,363,838
7,7,344,807
8,8,419,924
9,9,354,943


#### Query 11
##### Get the user with the most replies

Let's analyze another Neo4j query step by step:

1. `MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet)` - This line matches all users who have tweeted original tweets and also the tweets they have tweeted.

2. `WITH u, SUM(t.reply_count) AS total_replies` - This line groups the results by user and calculates the total number of replies on their original tweets.

3. `RETURN u.userid as UserID, u.username AS Username, total_replies as Total_Replies` - This line returns the user id, username, and the total number of replies for each user.

4. `ORDER BY total_replies DESC` - This line sorts the result in descending order by the total number of replies.

5. `LIMIT 1` - This line limits the number of results to just one, which means only the user with the highest number of total replies will be returned.

Therefore, this query returns the user with the highest number of total replies on their original tweets along with their user id and username.

Overall, this query is useful for identifying the most active users who receive a high number of replies on their tweets, which can be helpful for social media marketing and analysis purposes.

In [28]:
query = """
    MATCH (u:User)-[:TWEETED_ORIGINAL|RETWEETED|REPLIED|QUOTED]->(t:Tweet)
    WITH u, SUM(t.reply_count) AS total_replies
    RETURN u.userid as UserID, u.username AS Username, total_replies as Total_Replies
    ORDER BY total_replies DESC
    LIMIT 1
"""

result = session.run(query)
pd.DataFrame(result.data())

,UserID,Username,Total_Replies
0,3939574932,FightHaven,44611


#### Query 13
##### Get the top-20 hashtags that co-occur with the hashtag that has been used the most

We will design a query to find the most popular co-occurring hashtags with the most popular hashtag. Here is a step-by-step analysis of the query:

1. `MATCH (t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)` - Matches all tweets that have at least one hashtag and the hashtag nodes they are connected to.
2. `WITH h.hashtag AS main_hashtag, collect(DISTINCT t.tweet_id) AS main_tweet_ids` - Groups the tweets by their hashtags, collects the tweet IDs for each hashtag, and assigns the most popular hashtag to main_hashtag.
3. `ORDER BY size(main_tweet_ids) DESC LIMIT 1` - Orders the hashtags by the number of tweets that they are in, in descending order, and returns the most popular hashtag.
4. `MATCH (t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)` - Matches all tweets and hashtags again.
5. `WHERE h.hashtag <> main_hashtag AND t.tweet_id IN main_tweet_ids` - Filters out the tweets that contain the main hashtag and keeps only those tweets that have at least one hashtag in common with the main hashtag.
6. `WITH h.hashtag AS co_hashtag, count(DISTINCT t.tweet_id)` AS co_occurrences, main_hashtag - Groups the tweets by the co-occurring hashtags and counts the number of tweets in which they appear with the main hashtag. Also, assigns the main hashtag to a variable.
7. `ORDER BY co_occurrences DESC LIMIT 20` - Orders the co-occurring hashtags by the number of occurrences in descending order and returns only the top 20.
8. `RETURN main_hashtag, co_hashtag, co_occurrences` - Returns the main hashtag, the co-occurring hashtags, and their counts.

In [29]:
result = session.run("""
    MATCH (t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)
    WITH h.hashtag AS main_hashtag, collect(DISTINCT t.tweet_id) AS main_tweet_ids
    ORDER BY size(main_tweet_ids) DESC LIMIT 1
    MATCH (t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)
    WHERE h.hashtag <> main_hashtag AND t.tweet_id IN main_tweet_ids
    WITH h.hashtag AS co_hashtag, count(DISTINCT t.tweet_id) AS co_occurrences, main_hashtag
    ORDER BY co_occurrences DESC LIMIT 20
    RETURN main_hashtag, co_hashtag, co_occurrences
    """)

pd.DataFrame(result.data())

,main_hashtag,co_hashtag,co_occurrences
0,womenintech,coding,612
1,womenintech,javascript,602
2,womenintech,tech,574
3,womenintech,programming,529
4,womenintech,womenwhocode,495
5,womenintech,100DaysOfCode,460
6,womenintech,DEVCommunity,437
7,womenintech,100daysofcode,431
8,womenintech,technology,426
9,womenintech,python,421


#### Query 15
##### Get the most "important" user in the dataset (use Graph algorithms: Pagerank, Betweenness centrality, etc). You will apply these algorithms in the mention network (which includes retweets).

In [49]:
# Query 15 - Most important User
drop_query = """CALL gds.graph.drop('pageRankGraph') YIELD graphName;"""
session.run(drop_query)

create_graph_query = """
CALL gds.graph.project(
  'pageRankGraph',
  'User',
  {
    MENTIONED: {
      type: 'MENTIONED',
      orientation: 'NATURAL'
    }
  }
)
"""

pagerank_query = """
CALL gds.pageRank.stream('pageRankGraph')
YIELD nodeId, score
WITH nodeId, score
ORDER BY score DESC
MATCH (u:User)
WHERE id(u) = nodeId
WITH u.username AS username, MAX(score) AS max_score
RETURN DISTINCT username, max_score
"""

session.run(create_graph_query)
result = session.run(pagerank_query)
display(pd.DataFrame(result.data()))

,username,max_score
0,cryptosanthoshK,189.487500
1,Khulood_Almani,122.873344
2,GirlsWhoCode,122.225949
3,CatherineAdenle,111.718941
4,drlaxmanbjp,106.087470
...,...,...
18058,mandanaIR_,0.150000
18059,jijothelappilly,0.150000
18060,Damibanjo2,0.150000
18061,micanyonge,0.150000


#### Query 17
##### For the 5th most important user, get the list of hashtags and URLs that have been posted (if no hashtags or URLs - check another user e.g. 6th, 7th , etc..)

In [42]:
query = """
MATCH (u1:User)-[:USED_HASHTAG]->(u1_hashtag:Hashtag)
WHERE u1.username = "CatherineAdenle"
RETURN DISTINCT u1_hashtag.hashtag AS output

UNION ALL

MATCH (u1:User)-[:USED_URL]->(u1_url:URL)
WHERE u1.username = "CatherineAdenle"
RETURN DISTINCT u1_url.url AS output
"""

result = session.run(query)
print(np.array(result.values()).ravel())

['DataScience' 'NLP' 'Cloud' 'CyberSecurity' 'womenintech' 'TensorFlow'
 'IoT' 'Fintech' 'flutter' '100DaysOfCode' 'coding' 'MachineLearning' 'AI'
 'Programming' 'javascript' 'Python' 'Analytics' 'Business' 'Bigdata'
 'Data' 'TechNews' 'technology' '100DaysOfMLCode' 'DEVCommunity'
 'programming' 'ML' 'ethics' 'artificalintelligence' 'Microsoft'
 'innovation' 'techtwitter' 'tech' 'PredictiveProgramming'
 'ArtificialIntelligence' 'ChatGPTGOD' 'ChatGPT' 'BigData' 'fintech'
 'CloudComputing' 'machinelearning' 'productengineering' 'UI' 'UX'
 'productdesign' '5G' 'CES' 'BlackTechTwitter' 'mobile' 'Gadget'
 'prototype' 'phone' 'Samsung' 'CES2023' 'MWC2023' 'Electronic' 'gadget'
 'VR' 'wearables' 'autonomous' 'chatgpt' 'metaverse' '100daysofcode'
 'devcommunity' 'Coding' 'java' 'IIoT' 'Tech' 'Rstats' 'DataScientists'
 'DeepLearning' 'jobs' 'CodeT5' 'JavaScript' 'codenewbies' 'Womenintech'
 'openAI' '100daysofCoding' 'coders' 'GitHub' 'automation'
 'artificialIntelligence' 'datasets' 'CheatShee

#### Query 19
##### Get the user communities that have been created based on the users' interactions and visualize them (Louvain algorithm).

In [36]:
# drop_query = """CALL gds.graph.drop('louvainGraph') YIELD graphName;"""
# session.run(drop_query)

create_graph_query = """
CALL gds.graph.project(
  'louvainGraph',
  ['User'],
  ['MENTIONED', 'TWEETED_ORIGINAL', 'USED_HASHTAG', 'USED_URL']
)
"""

louvain_query = """
CALL gds.louvain.write('louvainGraph', {writeProperty: 'community'})
YIELD communityCount, modularity, modularities
"""
session.run(create_graph_query)
result = session.run(louvain_query)
communities = pd.DataFrame(result.data())
communities

,communityCount,modularity,modularities
0,4007,0.782913,"[0.7245289915561008, 0.7684591768066569, 0.776..."


#### Extra Query 1
##### Get the top-10 users who posted the most tweets during the year 2022. In order to count a tweet, it should contain at least one of the top-5 mostly used hashtags of all time.

In [37]:
query = """
    MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)
    WITH h, count(DISTINCT h) AS hashtag_count
    ORDER BY hashtag_count DESC
    LIMIT 5
    WITH collect(h) AS top_hashtags
    MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet)-[:HAS_HASHTAG]->(h:Hashtag)
    WHERE h IN top_hashtags AND datetime(t.created_at) >= datetime('2022-01-01T00:00:00.000Z') AND datetime(t.created_at) < datetime('2023-01-01T00:00:00.000Z')
    WITH u, count(DISTINCT t) AS tweet_count
    ORDER BY tweet_count DESC
    LIMIT 10
    RETURN u.username, tweet_count
"""

result = session.run(query)
pd.DataFrame(result.data())

,u.username,tweet_count
0,Khulood_Almani,65
1,WomenTechNet,37
2,Eli_Krumova,20
3,alod83,13
4,CatherineAdenle,5
5,WomenWhoCode,5
6,Manish_kumar3_1,3
7,Strategy_Gal,3
8,hertechtrail,3
9,WestArt_Factory,3


#### Extra Query 2
##### Get the users which have minimum average tweets per day bigger than the average number of tweets of a user per day. Only users with more that 2 tweets per day will be used in order to calculate the average number of tweets per day.

In [38]:
query = """
MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet) 
WITH u, count(t) as numTweets, date(datetime(t.created_at)) as tweetDate
WITH u, numTweets, min(tweetDate) as minTweetDate, max(tweetDate) as maxTweetDate
WITH u, numTweets, duration.between(minTweetDate, maxTweetDate).days + 1 as numDays
WITH u, numTweets / toFloat(numDays) as tweetsPerDay
WITH u, min(tweetsPerDay) as minTweetsPerDay
CALL {
    MATCH (u:User)-[:TWEETED_ORIGINAL]->(t:Tweet)
    WITH u, count(t) as numTweets, date(datetime(t.created_at)) as tweetDate
    WITH u, numTweets, min(tweetDate) as minTweetDate, max(tweetDate) as maxTweetDate
    WITH u, numTweets, duration.between(minTweetDate, maxTweetDate).days + 1 as numDays
    WITH u, numTweets / toFloat(numDays) as tweetsPerDay
    WHERE numTweets > 1
    WITH avg(tweetsPerDay) as totalAvgTweetsPerDay
    RETURN totalAvgTweetsPerDay
}
with minTweetsPerDay, totalAvgTweetsPerDay, u.username as username
WHERE minTweetsPerDay > totalAvgTweetsPerDay
RETURN username, minTweetsPerDay
ORDER BY minTweetsPerDay DESC
"""

result = session.run(query)
pd.DataFrame(result.data())

,username,minTweetsPerDay
0,VivianAaron5,71.0
1,mrflean,5.0
2,ColorsTV,5.0
3,fabinsanity,4.0
4,Help4uHealthy,4.0
5,Freelancerkelvn,4.0
6,HackwithNichola,4.0
